## L3: Multi-agent Customer Support Automation
In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
# !pip install crewai crewai_tools langchain_community langchain-openai

In [2]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os


load_dotenv()

default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

## Role Playing, Focus and Cooperation

In [3]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
    llm=default_llm,
	verbose=True
)

By not setting allow_delegation=False, allow_delegation takes its default value of being True.
This means the agent can delegate its work to another agent which is better suited to do a particular task.

In [4]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    llm=default_llm,
	verbose=True
)

- Role Playing: Both agents have been given a role, goal and backstory.
- Focus: Both agents have been prompted to get into the character of the roles they are playing.
- Cooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

In [5]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.
  search_tool = SerperDevTool()
  scrape_tool = ScrapeWebsiteTool()

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [6]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

Different Ways to Give Agents Tools:
- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.
Note: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [7]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

MS, Note - the task assumes completeness/reasonability of the request (it can be "hello"), it kind of doesn't assume there's a dialog. Let's see how it goes further

`quality_assurance_review` is not using any Tool(s). Here the QA Agent will only review the work of the Support Agen

In [8]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

### Creating the Crew


Setting memory=True when putting the crew together enables Memory.

In [9]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True,
  embedder={  
    "provider": "azure_openai",
    "config": {
        "model": "text-embedding-ada-002", 
        "deployment_name": "text-embedding-ada-002",
        # "api_base": os.environ.get("AZURE_OPENAI_ENDPOINT"),
        "api_key": os.environ.get("AZURE_OPENAI_KEY")
      }
  }
)

# There's a bug that doesn't allow to configure the embedding for Azure OpenAI

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


### Running the Crew
Note: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.
Guardrails - by running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [10]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Thought: To provide accurate guidance to Andrew Ng from DeepLearningAI on setting up a Crew and adding memory to it, I need to consult the specific documentation provided by crewAI. This will ensure my response is complete and precise according to the latest available information.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"}
 

Assembling and Activating Your CrewAI Team - crewAI
 Skip t

As of May 2024 "memory=True" imlicitly depends on OpenAI embeddings and embedchain, need special config in .env

In [11]:
from IPython.display import Markdown
Markdown(result)

Hi Andrew Ng,

Thank you for reaching out to us at crewAI! I'm delighted to assist you with setting up a Crew and adding memory to it. Here’s a step-by-step guide to help you get started and ensure your crew is equipped with memory capabilities. Additionally, I'll address some specific questions and concerns you might have.

### Step-by-Step Guide to Setting Up a Crew and Adding Memory

#### Step 0: Installation
First, you need to install CrewAI and any necessary packages for your project. Ensure you have Python version >=3.10 and <=3.13.

```bash
pip install crewai
pip install 'crewai[tools]'
```

#### Step 1: Assemble Your Agents
Define your agents with distinct roles, backstories, and enhanced capabilities like verbose mode and memory usage. Here’s an example of creating agents with memory:

```python
import os
from crewai import Agent
from crewai_tools import SerperDevTool

# Set your API keys
os.environ["SERPER_API_KEY"] = "Your Key" 
os.environ["OPENAI_API_KEY"] = "Your Key"

# Initialize tools
search_tool = SerperDevTool()

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
    role='Senior Researcher',
    goal='Uncover groundbreaking technologies in {topic}',
    verbose=True,
    memory=True,
    backstory=(
        "Driven by curiosity, you're at the forefront of"
        "innovation, eager to explore and share knowledge that could change"
        "the world."
    ),
    tools=[search_tool],
    allow_delegation=True
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
    role='Writer',
    goal='Narrate compelling tech stories about {topic}',
    verbose=True,
    memory=True,
    backstory=(
        "With a flair for simplifying complex topics, you craft"
        "engaging narratives that captivate and educate, bringing new"
        "discoveries to light in an accessible manner."
    ),
    tools=[search_tool],
    allow_delegation=False
)
```

#### Step 2: Define the Tasks
Specify the objectives for your agents. Here’s how to define tasks:

```python
from crewai import Task

# Research task
research_task = Task(
    description=(
        "Identify the next big trend in {topic}."
        "Focus on identifying pros and cons and the overall narrative."
        "Your final report should clearly articulate the key points,"
        "its market opportunities, and potential risks."
    ),
    expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
    tools=[search_tool],
    agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
    description=(
        "Compose an insightful article on {topic}."
        "Focus on the latest trends and how it's impacting the industry."
        "This article should be easy to understand, engaging, and positive."
    ),
    expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
    tools=[search_tool],
    agent=writer,
    async_execution=False,
    output_file='new-blog-post.md'  # Example of output customization
)
```

#### Step 3: Form the Crew
Combine your agents into a crew and set the workflow process. Make sure to include memory in the configuration:

```python
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.sequential,  # Optional: Sequential task execution is default
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True
)
```

#### Step 4: Kick It Off
Initiate the process with your enhanced crew ready. Input variables will be interpolated into the agents and tasks for a personalized approach:

```python
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
print(result)
```

### Additional Information and Troubleshooting

#### API Integration Issues
If you encounter difficulties integrating the crewAI API with your existing systems, here’s a troubleshooting guide:

1. **Check API Key Configuration**: Ensure your API keys are correctly set in your environment variables.
2. **Verify Dependencies**: Make sure all required packages are installed.
3. **Error Messages**: If you see specific error messages, refer to our [API documentation](https://docs.crewai.com/api) for detailed explanations and solutions.

#### Customizing Memory Capabilities
To customize memory capabilities for different team roles, follow these best practices:

- **Define Clear Objectives**: Tailor memory settings based on the specific needs of each role.
- **Use Templates**: Utilize available templates for common roles and customize them as needed.
- **Example**
  ```python
  researcher.memory_settings = {'max_memory_size': 1000, 'memory_retention': 'long-term'}
  writer.memory_settings = {'max_memory_size': 500, 'memory_retention': 'short-term'}
  ```

#### Training and Onboarding Support
We offer various training sessions and onboarding support to help your team effectively utilize crewAI's features. Here are some resources:

- **Webinars**: Join our upcoming webinars for live demonstrations and Q&A sessions.
- **Documentation**: Access comprehensive guides and tutorials on our [documentation page](https://docs.crewai.com).
- **One-on-One Support**: Contact us to schedule personalized training sessions.

#### Performance Metrics and Reporting
To track and interpret performance metrics:

1. **Access Metrics**: Navigate to the analytics section in the crewAI dashboard.
2. **Key Metrics**: Monitor metrics such as task completion time, agent performance, and system usage.
3. **Reporting**: Generate reports to analyze trends and improve your workflow.

#### Troubleshooting Common Issues
For common issues like slow response times and system crashes:

- **Response Times**: Optimize your agents' configurations and reduce the number of concurrent tasks.
- **System Crashes**: Ensure your system meets the required specifications and update to the latest version of crewAI.
- **Support**: If issues persist, contact our support team for further assistance.

### Conclusion
By following these steps and addressing the additional points, you'll be able to set up your CrewAI team with memory capabilities effectively and troubleshoot any potential issues. This setup not only allows your agents to retain information across tasks but also enhances their overall performance and collaboration.

If you have any more questions or need further assistance, feel free to reach out. I'm here to help!

Best regards,
[Your Name]
Senior Support Representative
crewAI